In [ ]:
import pandas as pd

In [ ]:
# Importa i dati dal file excel
#Modifica la precisione nella visualizzazione delle cifre
pd.options.display.precision = 10

#Legge il csv e lo stampa
df_DatiSensori = pd.read_excel('Dati-gruppo1.xlsx')

#Mostra tutte le colonne
# pd.option_context('display.max_rows', df_DatiSensori.shape[0]+1)
df_DatiSensori

In [ ]:
# Query 1

pd.set_option('display.max_rows', 100)
# query effettuata con un unico comando
# df_Benzene = df_DatiSensori.sort_values(by='C6H6_ug/m3', ascending =False)[['postazione', 'Data', 'C6H6_ug/m3']].head(100)

# creazione di un dataFrame ordinato in modo discendente per i valori di benzene
df_Benzene = df_DatiSensori.sort_values(by='C6H6_ug/m3', ascending =False).drop_duplicates(subset=['Data'])
# creazione di un dataFrame contenente i 100 maggiori valori di benzene
query_Benzene = df_Benzene[['postazione', 'Data', 'C6H6_ug/m3']].head(100)
query_Benzene

In [ ]:
# Query 2 per il sensore H2S

# query effettuata con un unico comando
# df_H2S = df_DatiSensori.sort_values(by='H2S_ug/m3', ascending =False)[['postazione', 'Data', 'H2S_ug/m3']].head(100)
# creazione di un dataFrame ordinato in modo discendente per i valori di acido solfidrico del sensore H2S
df_H2S = df_DatiSensori.sort_values(by='H2S_ug/m3', ascending =False).drop_duplicates(subset=['Data'])
# creazione di un dataFrame contenente i 100 maggiori valori di acido solfidrico del sensore H2SJ
query_H2S = df_H2S[['postazione', 'Data', 'H2S_ug/m3', 'H2SJ_ug/m3']].head(100)
query_H2S

In [ ]:
# Query 2 per il sensore H2SJ

# query effettuata con un unico comando
# df_H2SJ = df_DatiSensori.sort_values(by='H2SJ_ug/m3', ascending =False)[['postazione', 'Data', 'H2SJ_ug/m3']].head(100)
# creazione di un dataFrame ordinato in modo discendente per i valori di acido solfidrico del sensore H2SJ
df_H2SJ = df_DatiSensori.sort_values(by='H2SJ_ug/m3', ascending =False).drop_duplicates(subset=['Data'])
# creazione di un dataFrame contenente i 100 maggiori valori di acido solfidrico del sensore H2SJ
query_H2SJ = df_H2SJ[['postazione', 'Data', 'H2SJ_ug/m3']].head(100)
query_H2SJ

In [ ]:
# Query 3 per il sensore VOC

# query effettuata con un unico comando
# df_VOC = df_DatiSensori.sort_values(by='VOC_ppm', ascending =True)[['postazione', 'Data', 'VOC_ppm']].head(100)
# creazione di un dataFrame ordinato in modo discendente per i valori del sensore VOC
df_VOC = df_DatiSensori.sort_values(by='VOC_ppm', ascending =True).drop_duplicates(subset=['Data'])
# creazione di un dataFrame contenente i 100 più bassi valori del sensore VOC
query_VOC = df_VOC[['postazione', 'Data', 'VOC_ppm']].head(100)
query_VOC

In [ ]:
# Query 3 per il sensore PIDVOC

# query effettuata con un unico comando
# df_PIDVOC = df_DatiSensori.sort_values(by='PIDVOC_ppb', ascending =True)[['postazione', 'Data', 'PIDVOC_ppb']].head(100)
# creazione di un dataFrame ordinato in modo discendente per i valori del sensore PIDVOC
df_PIDVOC = df_DatiSensori.sort_values(by='PIDVOC_ppb', ascending =True).drop_duplicates(subset=['Data'])
# creazione di un dataFrame contenente i 100 più bassi valori del sensore PIDVOC
query_PIDVOC = df_PIDVOC[['postazione', 'Data', 'PIDVOC_ppb']].head(100)
query_PIDVOC

In [ ]:
# query 4: dividere il dataset in gruppi di ore consecutive, sommare le ore e calcolare la media per ogni ora 
# e quella più bassa